# 재사용 가능한 @property 메서드에는 디스크립터를 사용하자

Python에 내장된 `@property`의 큰 문제점은 재사용성이다. 다시 말해, `@property`로 데코레이트하는 메서드를 같은 클래스에 속한 여러 속성에 사용하지 못한다. 또한 관련 없는 클래스에서도 재사용할 수 없다.

In [3]:
class Homework(object):
    def __init__(self):
        self._grade = 0
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

`@property`를 사용하면 이 클래스를 쉽게 사용할 수 있다.

In [4]:
galileo = Homework()
galileo.grade = 95

학생들의 시험 성적을 매긴다고 해보자. 시험은 여러 과목으로 구성되어 있고 과목별로 점수가 있다.

이 코드는 금방 장황해진다. 시험 영역마다 새 `@property`와 관련 검증이 필요하다.

In [6]:
class Exam(object):
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._write_grade = value
        
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

이런 방법은 범용으로 사용하기에 좋지 않다. 과제와 시험 이외의 항목에도 이 백분율 검증을 재사용하고 싶다면 `@property`와 `_check_grade`를 반복적으로 작성해야 한다.

Python에서 이런 작업을 할 때 더 좋은 방법은 descriptor를 사용하는 것이다. descriptor protocol은 속성에 대한 접근을 언어에서 해석할 방법을 정의한다. discriptor class는 반복 코드 없이도 성적 검증 동작을 재사용할 수 있게 해주는 `__get__`과 `__set__` 메서드를 제공할 수 있다. 이런 목적으로 discriptor가 Mixin보다 좋은 방법이다.descriptor를 사용하면 한 클래스의 서로 다른 많은 속성에 같은 로직을 재사용할 수 있기 때문이다.

In [21]:
class Grade(object):
    def __get__(*args, **kwargs):
        pass
    def __set__(*args, **kwargs):
        pass


        
class Exam(object):
    # 클래스 속성
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [22]:
exam = Exam()
exam.writing_grade = 40

위 코드는 다음과 같이 해석된다.

In [23]:
Exam.__dict__['writing_grade'].__set__(exam, 40)

이번에는 다음과 같이 프로퍼티를 얻어온다고 하자.

In [24]:
print(exam.writing_grade)

None


위 코드는 다음과 같이 해석된다.

In [25]:
print(Exam.__dict__['writing_grade'].__get__(exam, Exam))

None


이렇게 동작하게 만드는 건 `object`의 `__getattribute__` 메서드다. 간략히 말하면 Exam 인스턴스에 `writing_grade` 속성이 없으면 Python은 대신 Exam 클래스의 속성을 이용한다. 이 클래스의 속성이 `__get__`과 `__set__` 메서드를 갖춘 객체라면 Python은 descriptor protocol을 따른다고 가정한다.

In [28]:
class Grade(object):
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value
        
class Exam(object):
    # 클래스 속성
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

불행히도 위 코드는 잘못 구현되어 있어서 제대로 동작하지 않는다.

한 Exam 인스턴스에 있는 여러 속성에 접근하는 것은 기대한 대로 동작한다.

In [29]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


여러 Exam 인스턴스의 이런 속성에 접근 시 기대하지 않은 동작을 하게 된다.

In [30]:
second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('First', first_exam.writing_grade, 'is wrong')

Second 75 is right
First 75 is wrong


문제는 한 Grade 인스턴스가 모든 `writing_grade` 클래스 속성으로 공유된다는 점이다. 이 속성에 대응하는 `Grade` 인스턴스는 프로그램에서 Exam 인스턴스를 생성할 때마다 생성되는 게 아니라 Exam 클래스를 처음 정의할 때 한 번만 생성된다.

이를 해결하기 위해 각 Exam 인스턴스별로 값을 추적하는 Grade 클래스가 필요하다. 여기서는 dictionary에 각 인스턴스의 상태를 저장하는 방법으로 값을 추적한다.

In [31]:
class Grade(object):
    def __init__(self):
        self._values = {}
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

하지만 메모리 누수라는 문제점이 있다. `_values` dictionary는 프로그램의 수명 동안 `__set__`에 전달된 모든 Exam 인스턴스의 참조를 저장한다. 결국 인스턴스의 참조 개수가 절대로 0이 되지 않아 가비지 컬렉터가 정리하지 못하게 된다.

Python의 내장 모듈 `weakref`를 사용하면 이 문제를 해결할 수 있다.

`_values`에 사용한 간단한 dictionary를 대체할 수 있는 `WeakKeyDictionary`라는 특별한 클래스를 제공한다.

이 모듈은 런타임에 마지막으로 남은 `Exam` 인스턴스의 참조를 갖고 있다는 사실을 알면 키 집합에서 Exam 인스턴스를 제거하는 것이다. Python이 대신 참조를 관리해주고 모든 Exam 인스턴스가 더는 사용되지 않으면 `_values` 딕셔너리가 비어있게 한다.

In [35]:
from weakref import WeakKeyDictionary

class Grade(object):
    def __init__(self):
        self._values = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

In [36]:
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [37]:
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print('First', first_exam.writing_grade, 'is right')
print('Second', second_exam.writing_grade, 'is right')

First 82 is right
Second 75 is right
